In [59]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import numpy as np
import pandas as pd
import regex as re

In [60]:
chromedriver = "chromedriver.exe" # path to the chromedriver executable
chromedriver = os.path.expanduser(chromedriver)
#print('chromedriver path: {}'.format(chromedriver))
sys.path.append(chromedriver)
driver = webdriver.Chrome(chromedriver)

In [61]:
# Create your URL variable and assign the page range
# The page parsmeter in the URL is s=120 and so on, s=240... 
apt_url = "https://sfbay.craigslist.org/search/sfc/apa?"
pageRange = 120
listings = []
# driver.get(apt_url)

In [69]:
# Loop through the different pages 
for i in range(0,25):
    r = pageRange * i
    driver.get(apt_url + str(r))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    listings.append(soup.find_all('a', class_="result-title hdrlnk"))
#print(listings)

In [70]:
# results are returned as an iterable list
list_results = []

for link in listings:
    results = soup.find_all('li', class_="result-row")

# Loop through returned results
for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        title = result.find('a', class_="result-title").text
        # Identify and return price of listing
        price = result.a.span.text.strip('$')
        # Identify number of rooms
        n_rooms = result.find('span', class_ = 'housing').text.split()
        # Identify and return link to listing
        link = result.a['href']
        # Identify posting date
        post_time = result.find('time')['datetime']
        post_time = pd.to_datetime(post_time)
        # Identify neighborhood
        neighborhood = result.find('span', class_= 'result-hood').text
        
        list_results.append({'Rooms_Sqft': n_rooms, 'Price': price,
                        'Post_Title': title, 'URL': link, 'Post_Time': post_time, 'Neighborhood': neighborhood})
        
        # Print results only if title, price, and link are available
        if (title and price and link):
            print('-------------')
            print(title)
            print(price)
            print(n_rooms)
            print(link)
            print(post_time)
            print(neighborhood)
    except AttributeError as e:
        print(e)

-------------
LUMINA 1BED 1BATH W/PARKING Unit 2B
5200
['1br', '-', '1100ft2', '-']
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-lumina-1bed-1bath-parking/6817988643.html
2019-02-16 08:40:00
 (SOMA / south beach)
-------------
LUXURY 333 Bush Furnished - Financial District-avail May 1
6995
['1br', '-', '1050ft2', '-']
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-luxury-333-bush-furnished/6817988558.html
2019-02-16 08:40:00
 (financial district)
-------------
MODERN LUXURY 301 Mission, 1bed 1bath Fully Furnished
6500
['1br', '-']
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-modern-luxury-301-mission/6817988469.html
2019-02-16 08:40:00
 (financial district)
-------------
LUMINA 23rd floor FURNISHED 2/bed 2bath
8500
['2br', '-']
https://sfbay.craigslist.org/sfc/apa/d/san-francisco-lumina-23rd-floor/6817988329.html
2019-02-16 08:40:00
 (SOMA / south beach)
-------------
top flr/VIEW OF THE BAY/DOWNTOWN/1275 Clay/nob hill/parking!
3195
['1br', '-']
https://sfbay.crai

In [71]:
# Create DataFrame from the scraping results
apt_df = pd.DataFrame(list_results)

# Extract the bedrooms count from the Rooms_Sqft column
bedroom = apt_df['Rooms_Sqft']
rooms = []

for room in bedroom: # remove the "br" text to only show the bedroom number
    rooms.append(room[0].strip('br'))

apt_df['Bedrooms'] = rooms # create new column for room #
apt_df['Post_Time'] = apt_df['Post_Time'].dt.date # convert the posting time to be datetime format

apt_df

,Neighborhood,Post_Time,Post_Title,Price,Rooms_Sqft,URL,Bedrooms
0,(SOMA / south beach),2019-02-16,LUMINA 1BED 1BATH W/PARKING Unit 2B,5200,"[1br, -, 1100ft2, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1
1,(financial district),2019-02-16,LUXURY 333 Bush Furnished - Financial District...,6995,"[1br, -, 1050ft2, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1
2,(financial district),2019-02-16,"MODERN LUXURY 301 Mission, 1bed 1bath Fully Fu...",6500,"[1br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1
3,(SOMA / south beach),2019-02-16,LUMINA 23rd floor FURNISHED 2/bed 2bath,8500,"[2br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2
4,(nob hill),2019-02-16,top flr/VIEW OF THE BAY/DOWNTOWN/1275 Clay/nob...,3195,"[1br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1
5,(SOMA / south beach),2019-02-16,"LUMINA 318 MAIN 2C 1/bed, den 2baths",5300,"[1br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1
6,(downtown / civic / van ness),2019-02-16,Studio/Downtown/755 O'farrell&Hyde/Remodeled/A...,2195,"[384ft2, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,384ft2
7,(russian hill),2019-02-16,Russian Hill/1320Lombard&Polk/W/D in unit/Remo...,4795,"[2br, -, 855ft2, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2
8,(downtown / civic / van ness),2019-02-16,Lower nob hill/1424 Polk/Central Location/W/D ...,3995,"[2br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2
9,(downtown / civic / van ness),2019-02-16,Studio/Downtown/755 O'farrell&Hyde/Remodeled/A...,2195,"[384ft2, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,384ft2


In [72]:
# Some of the returned listings don't have # of bedrooms listed and only sqft
# I want to remove any of these listings so that # bedrooms is a constant variable for each row
apt_new = apt_df[~apt_df.Bedrooms.str.contains('ft2')]
apt_new

,Neighborhood,Post_Time,Post_Title,Price,Rooms_Sqft,URL,Bedrooms
0,(SOMA / south beach),2019-02-16,LUMINA 1BED 1BATH W/PARKING Unit 2B,5200,"[1br, -, 1100ft2, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1
1,(financial district),2019-02-16,LUXURY 333 Bush Furnished - Financial District...,6995,"[1br, -, 1050ft2, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1
2,(financial district),2019-02-16,"MODERN LUXURY 301 Mission, 1bed 1bath Fully Fu...",6500,"[1br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1
3,(SOMA / south beach),2019-02-16,LUMINA 23rd floor FURNISHED 2/bed 2bath,8500,"[2br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2
4,(nob hill),2019-02-16,top flr/VIEW OF THE BAY/DOWNTOWN/1275 Clay/nob...,3195,"[1br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1
5,(SOMA / south beach),2019-02-16,"LUMINA 318 MAIN 2C 1/bed, den 2baths",5300,"[1br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1
7,(russian hill),2019-02-16,Russian Hill/1320Lombard&Polk/W/D in unit/Remo...,4795,"[2br, -, 855ft2, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2
8,(downtown / civic / van ness),2019-02-16,Lower nob hill/1424 Polk/Central Location/W/D ...,3995,"[2br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2
10,(richmond / seacliff),2019-02-16,Charming Garden studio-Newly renovated -Free U...,2300,"[1br, -, 600ft2, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1
11,(Inner Richmond),2019-02-16,FREE 1st Month's Rent Inner Richmond 1bd/1ba S...,2200,"[1br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1


In [73]:
hoods = apt_new["Neighborhood"].unique()
hoods

array([' (SOMA / south beach)', ' (financial district)', ' (nob hill)',
       ' (russian hill)', ' (downtown / civic / van ness)',
       ' (richmond / seacliff)', ' (Inner Richmond)', ' (haight ashbury)',
       ' (excelsior / outer mission)', ' (north beach / telegraph hill)',
       ' (pacific heights)', ' (San Francisco - Downtown)',
       ' (sunset / parkside)', ' (inner richmond)',
       ' (twin peaks / diamond hts)', ' (marina / cow hollow)',
       ' (lower pac hts)', ' (castro / upper market)', ' (lower nob hill)',
       ' (laurel hts / presidio)', ' (mission district)',
       ' (alamo square / nopa)', ' (San Francisco - Alamo Square)',
       ' (Dublin, CA 94568)', ' (noe valley)', ' (El Cerrito)',
       ' (San Francisco)', ' (portola district)', ' (Central Sunset)',
       ' (bayview)'], dtype=object)

In [74]:
# Convert price and # rooms to int in order to graph data
apt_new['Price'] = apt_new['Price'].astype(int)
apt_new['Bedrooms'] = apt_new['Bedrooms'].astype(int)


##############################################################

# Clean up the Neighborhood name
apt_new['Hood'] = apt_new['Neighborhood'].str.extract(r"\((.*?)\)", expand=False)
apt_new['Hood'] = apt_new['Hood'].astype(str)
apt_new['Hood'] = apt_new['Hood'].str.strip()

neighborhoods = apt_new["Hood"].unique()
new_hoods = ['SOMA', 'FIDI', 'Nob Hill', 'Russian Hill', 'Van Ness', 'Richmond/Seacliff', 'Inner Richmond', 'Haight Ashbury', 'Outer Mission', 'North Beach', 'Pac Heights', 'Downtown', 'Sunset', 'Inner Richmond', 
             'Twin Peacks', 'Marina', 'Lower Pac Heights', 'Castro', 'Lower Nob Hill', 'Presidio', 'Mission', 'Nopa', 
             'Alamo Square', 'Dublin', 'Noe Vallet', 'El Cerrito', 'San Francisco', 'Portola District', 'Central Sunset', 'Bayview']
my_dict = dict(zip(neighborhoods, new_hoods))

apt_new['Hood'].replace(my_dict, inplace=True)


apt_new


C:\Users\amand\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\amand\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\amand\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,Neighborhood,Post_Time,Post_Title,Price,Rooms_Sqft,URL,Bedrooms,Hood
0,(SOMA / south beach),2019-02-16,LUMINA 1BED 1BATH W/PARKING Unit 2B,5200,"[1br, -, 1100ft2, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1,SOMA
1,(financial district),2019-02-16,LUXURY 333 Bush Furnished - Financial District...,6995,"[1br, -, 1050ft2, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1,FIDI
2,(financial district),2019-02-16,"MODERN LUXURY 301 Mission, 1bed 1bath Fully Fu...",6500,"[1br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1,FIDI
3,(SOMA / south beach),2019-02-16,LUMINA 23rd floor FURNISHED 2/bed 2bath,8500,"[2br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2,SOMA
4,(nob hill),2019-02-16,top flr/VIEW OF THE BAY/DOWNTOWN/1275 Clay/nob...,3195,"[1br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1,Nob Hill
5,(SOMA / south beach),2019-02-16,"LUMINA 318 MAIN 2C 1/bed, den 2baths",5300,"[1br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1,SOMA
7,(russian hill),2019-02-16,Russian Hill/1320Lombard&Polk/W/D in unit/Remo...,4795,"[2br, -, 855ft2, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2,Russian Hill
8,(downtown / civic / van ness),2019-02-16,Lower nob hill/1424 Polk/Central Location/W/D ...,3995,"[2br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2,Van Ness
10,(richmond / seacliff),2019-02-16,Charming Garden studio-Newly renovated -Free U...,2300,"[1br, -, 600ft2, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1,Richmond/Seacliff
11,(Inner Richmond),2019-02-16,FREE 1st Month's Rent Inner Richmond 1bd/1ba S...,2200,"[1br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1,Inner Richmond


In [75]:
apt_new['Source'] = 'Craigslist'
apt_new

C:\Users\amand\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Neighborhood,Post_Time,Post_Title,Price,Rooms_Sqft,URL,Bedrooms,Hood,Source
0,(SOMA / south beach),2019-02-16,LUMINA 1BED 1BATH W/PARKING Unit 2B,5200,"[1br, -, 1100ft2, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1,SOMA,Craigslist
1,(financial district),2019-02-16,LUXURY 333 Bush Furnished - Financial District...,6995,"[1br, -, 1050ft2, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1,FIDI,Craigslist
2,(financial district),2019-02-16,"MODERN LUXURY 301 Mission, 1bed 1bath Fully Fu...",6500,"[1br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1,FIDI,Craigslist
3,(SOMA / south beach),2019-02-16,LUMINA 23rd floor FURNISHED 2/bed 2bath,8500,"[2br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2,SOMA,Craigslist
4,(nob hill),2019-02-16,top flr/VIEW OF THE BAY/DOWNTOWN/1275 Clay/nob...,3195,"[1br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1,Nob Hill,Craigslist
5,(SOMA / south beach),2019-02-16,"LUMINA 318 MAIN 2C 1/bed, den 2baths",5300,"[1br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1,SOMA,Craigslist
7,(russian hill),2019-02-16,Russian Hill/1320Lombard&Polk/W/D in unit/Remo...,4795,"[2br, -, 855ft2, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2,Russian Hill,Craigslist
8,(downtown / civic / van ness),2019-02-16,Lower nob hill/1424 Polk/Central Location/W/D ...,3995,"[2br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2,Van Ness,Craigslist
10,(richmond / seacliff),2019-02-16,Charming Garden studio-Newly renovated -Free U...,2300,"[1br, -, 600ft2, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1,Richmond/Seacliff,Craigslist
11,(Inner Richmond),2019-02-16,FREE 1st Month's Rent Inner Richmond 1bd/1ba S...,2200,"[1br, -]",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1,Inner Richmond,Craigslist


In [76]:
craigs_df = apt_new[['Source', 'Hood', 'Bedrooms', 'Price', 'Post_Title', 'Post_Time', 'URL']]
craigs_df

,Source,Hood,Bedrooms,Price,Post_Title,Post_Time,URL
0,Craigslist,SOMA,1,5200,LUMINA 1BED 1BATH W/PARKING Unit 2B,2019-02-16,https://sfbay.craigslist.org/sfc/apa/d/san-fra...
1,Craigslist,FIDI,1,6995,LUXURY 333 Bush Furnished - Financial District...,2019-02-16,https://sfbay.craigslist.org/sfc/apa/d/san-fra...
2,Craigslist,FIDI,1,6500,"MODERN LUXURY 301 Mission, 1bed 1bath Fully Fu...",2019-02-16,https://sfbay.craigslist.org/sfc/apa/d/san-fra...
3,Craigslist,SOMA,2,8500,LUMINA 23rd floor FURNISHED 2/bed 2bath,2019-02-16,https://sfbay.craigslist.org/sfc/apa/d/san-fra...
4,Craigslist,Nob Hill,1,3195,top flr/VIEW OF THE BAY/DOWNTOWN/1275 Clay/nob...,2019-02-16,https://sfbay.craigslist.org/sfc/apa/d/san-fra...
5,Craigslist,SOMA,1,5300,"LUMINA 318 MAIN 2C 1/bed, den 2baths",2019-02-16,https://sfbay.craigslist.org/sfc/apa/d/san-fra...
7,Craigslist,Russian Hill,2,4795,Russian Hill/1320Lombard&Polk/W/D in unit/Remo...,2019-02-16,https://sfbay.craigslist.org/sfc/apa/d/san-fra...
8,Craigslist,Van Ness,2,3995,Lower nob hill/1424 Polk/Central Location/W/D ...,2019-02-16,https://sfbay.craigslist.org/sfc/apa/d/san-fra...
10,Craigslist,Richmond/Seacliff,1,2300,Charming Garden studio-Newly renovated -Free U...,2019-02-16,https://sfbay.craigslist.org/sfc/apa/d/san-fra...
11,Craigslist,Inner Richmond,1,2200,FREE 1st Month's Rent Inner Richmond 1bd/1ba S...,2019-02-16,https://sfbay.craigslist.org/sfc/apa/d/san-fra...


In [77]:
craigs_df.to_excel("Craigslist_Data.xlsx", index=False)